### Imports and Types

In [1]:
from abc import ABC, abstractmethod
from typing import TypeAlias
import numpy as np
import pandas as pd

u_mas: TypeAlias = float
"""Milliarcseconds"""
u_ly: TypeAlias = float
"""Light years"""
u_degrees: TypeAlias = float
"""Degrees"""
u_fc: TypeAlias = float
"""Fraction of the speed of light, 0 <= fc <= 1"""
u_years: TypeAlias = float
"""Years"""

_ = _

### Utility Functions and Destination Class

In [2]:
def parallax_to_distance(parallax: u_mas) -> u_ly:
    return (1 / (parallax / 1000)) * 3.262


def a(deg: float, min_: float, sec: float) -> float:
    return deg + min_ / 60 + sec / 3600


class Destination:
    def __init__(self,
                 name: str, messier_number: int | None, object_type: str,
                 parallax: u_mas, parallax_error: u_mas | None = None,
                 right_ascension: u_degrees = float("NaN"), declination: u_degrees = float("NaN"),
                 radius: u_ly | None = None):
        assert right_ascension == right_ascension, "Right ascension must be a number"
        assert declination == declination, "Declination must be a number"

        self.name = name
        self.messier_number = messier_number
        self.object_type = object_type
        self.parallax: u_mas = parallax
        self.parallax_error: u_mas | None = parallax_error

        self.distance_ly = parallax_to_distance(parallax)
        if parallax_error is not None:
            dist_a = parallax_to_distance(parallax + parallax_error)
            dist_b = parallax_to_distance(parallax - parallax_error)
            self.distance_ly_error = abs((dist_a - dist_b) / 2)
        else:
            self.distance_ly_error = None

        self.right_ascension = right_ascension
        self.declination = declination

        galactic_x = np.cos(np.radians(self.declination)) * np.cos(np.radians(self.right_ascension)) * self.distance_ly
        galactic_y = np.cos(np.radians(self.declination)) * np.sin(np.radians(self.right_ascension)) * self.distance_ly
        galactic_z = np.sin(np.radians(self.declination)) * self.distance_ly
        self.galactic_position: np.ndarray[3, np.dtype[np.float64]] = np.array((galactic_x, galactic_y, galactic_z), dtype=np.float64)
        """z is the direction of the north galactic pole"""

        self.radius: u_ly | None = radius
    
    def __str__(self):
        msr = f" (M{self.messier_number})" if self.messier_number else ""
        ly_err = f" ± {self.distance_ly_error:.3f}" if self.distance_ly_error else ""
        return f"{self.name}{msr} ({self.object_type}) - {self.distance_ly:.2f}{ly_err} ly"

    def __eq__(self, other):
        if self is other:
            return True
        if not isinstance(other, Destination):
            return False
        return (self.name == other.name                       and
                self.messier_number == other.messier_number   and
                self.object_type == other.object_type         and
                self.parallax == other.parallax               and
                self.parallax_error == other.parallax_error   and
                self.right_ascension == other.right_ascension and
                self.declination == other.declination)


destinations = [
    D_CRAB_NEBULA:=Destination("Crab Nebula", 1, "Planetary Nebula", 0.51104, 0.07876, right_ascension=a(5, 34, 31.94), declination=a(22, 0, 52.2), radius=5.5),
    D_LAGOON_NEBULA:=Destination("Lagoon Nebula", 8, "Emission Nebula", 0.8505, 0.0952, right_ascension=a(18, 3, 37.1), declination=a(-24, 23, 12), radius=20), # technically 55 by 20 ly
    D_RING_NEBULA:=Destination("Ring Nebula", 57, "Planetary Nebula", 1.2696, 0.0438, right_ascension=a(18, 53, 35.079), declination=a(33, 1, 45.03), radius=1.3),
    D_ALPHA_CENTAURI:=Destination("Alpha Centauri", None, "Star System", 750.81, 0.38, right_ascension=a(14, 39, 36.494), declination=a(-60, 50, 2.3737)),
    D_PSR_B1257_12:=Destination("PSR B1257+12", None, "Pulsar", 1.41, 0.08, right_ascension=a(13, 0, 1), declination=a(12, 40, 57)),
]

"""
Galactic Long/Radial Coordinates:

PSR B1257+12    311 degrees  2313 ly
Ring Nebula      63 degrees  2569 ly
Crab Nebula     184 degrees  6383 ly
Lagoon Nebula     6 degrees  3835 ly
Alpha Centauri  316 degrees  4.34 ly
"""

for i, dest in enumerate(destinations):
    dest.i = i

print("Destinations:")
for dest in destinations:
    print(f"\t{dest}")

Destinations:
	Crab Nebula (M1) (Planetary Nebula) - 6383.06 ± 1007.673 ly
	Lagoon Nebula (M8) (Emission Nebula) - 3835.39 ± 434.758 ly
	Ring Nebula (M57) (Planetary Nebula) - 2569.31 ± 88.744 ly
	Alpha Centauri (Star System) - 4.34 ± 0.002 ly
	PSR B1257+12 (Pulsar) - 2313.48 ± 131.685 ly


### Destination Plotting

In [6]:
import matplotlib
matplotlib.use('nbAgg')
import matplotlib.pyplot as plt
plt.close("all")

df = pd.DataFrame([{
    "Name": dest.name,
    "Messier Number": dest.messier_number,
    "Object Type": dest.object_type,
    "Parallax (mas)": dest.parallax,
    "Parallax Error (mas)": dest.parallax_error,
    "Distance (ly)": dest.distance_ly,
    "Distance Error (ly)": dest.distance_ly_error,
    "Right Ascension": dest.right_ascension,
    "Declination": dest.declination,
    "Galactic Position (ly)": dest.galactic_position
} for dest in destinations])

# Plot all destinations in 3D

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.set_title('Galactic Positions of Destinations')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
#ax.scatter(0, 0, 0, label="Earth")
for dest in destinations:
    ax.scatter(dest.galactic_position[0], dest.galactic_position[1], dest.galactic_position[2], label=dest.name)

# ensure everything is visible
x_extreme = max([abs(dest.galactic_position[0]) for dest in destinations]) * 1.2
y_extreme = max([abs(dest.galactic_position[1]) for dest in destinations]) * 1.2
z_extreme = max([abs(dest.galactic_position[2]) for dest in destinations]) * 1.2
ax.set_xlim(-x_extreme, x_extreme)
ax.set_ylim(-y_extreme, y_extreme)
ax.set_zlim(-z_extreme, z_extreme)

plt.legend()
plt.show()

df

<IPython.core.display.Javascript object>

,Name,Messier Number,Object Type,Parallax (mas),Parallax Error (mas),Distance (ly),Distance Error (ly),Right Ascension,Declination,Galactic Position (ly)
0,Crab Nebula,1.0,Planetary Nebula,0.51104,0.07876,6383.061991,1007.673304,5.575539,22.014500,"[5889.670041441251, 574.9486701159833, 2392.63..."
1,Lagoon Nebula,8.0,Emission Nebula,0.85050,0.09520,3835.390947,434.758450,18.060306,-23.613333,"[3341.107304336296, 1089.4807784234767, -1536...."
2,Ring Nebula,57.0,Planetary Nebula,1.26960,0.04380,2569.313170,88.744498,18.893078,33.029175,"[2038.0415628857713, 697.5024996171479, 1400.4..."
3,Alpha Centauri,NaN,Star System,750.81000,0.38000,4.344641,0.002199,14.660137,-59.166007,"[2.1543587776276976, 0.5635837111994803, -3.73..."
4,PSR B1257+12,NaN,Pulsar,1.41000,0.08000,2313.475177,131.684917,13.000278,12.682500,"[2199.180193308989, 507.7319834094088, 507.919..."


### Travelling Salesman

In [13]:
from itertools import permutations


class FakeEarthDestination:
    def __init__(self):
        self.i = float("NaN")
        self.name = "Earth"
        self.galactic_position = np.array((0, 0, 0), dtype=np.float64)

    def __str__(self):
        return "Earth"


EARTH = FakeEarthDestination()


def distance_between(dest_a: Destination, dest_b: Destination) -> u_ly:
    delta = dest_a.galactic_position - dest_b.galactic_position
    return np.linalg.norm(delta)


def total_distance(route: list[Destination]) -> float:
    # include distance from earth (0, 0, 0) to first destination and last destination back to earth
    return (distance_between(EARTH, route[0])
            + sum([distance_between(route[i], route[i + 1]) for i in range(len(route) - 1)])
            + distance_between(route[-1], EARTH))


def best_route(dests: list[Destination]) -> list[Destination]:
    br = None
    best_distance = float("inf")
    for route in permutations(dests):
        dist = total_distance(route)
        if dist < best_distance:
            best_distance = dist
            br = route
    return list(br)


best_route_ = [EARTH] + best_route(destinations) + [EARTH]
print("Best Route:")
for dest in best_route_:
    print(f"\t[{dest.i}] {dest}")


# Plot the best route

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.set_title('Galactic Positions of Destinations')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

longest_name = max([len(dest.name) for dest in best_route_])

for i in range(len(best_route_) - 1):
    dest = best_route_[i]
    next_dest = best_route_[i + 1]

    ax.plot([dest.galactic_position[0], next_dest.galactic_position[0]],
            [dest.galactic_position[1], next_dest.galactic_position[1]],
            [dest.galactic_position[2], next_dest.galactic_position[2]], label=f"•{dest.name} -> {next_dest.name}")
    ax.scatter(dest.galactic_position[0], dest.galactic_position[1], dest.galactic_position[2])

# ensure everything is visible
x_extreme = max([abs(dest.galactic_position[0]) for dest in destinations]) * 1.2
y_extreme = max([abs(dest.galactic_position[1]) for dest in destinations]) * 1.2
z_extreme = max([abs(dest.galactic_position[2]) for dest in destinations]) * 1.2

ax.set_xlim(-x_extreme, x_extreme)
ax.set_ylim(-y_extreme, y_extreme)
ax.set_zlim(-z_extreme, z_extreme)

plt.legend()
plt.show()


Best Route:
	[nan] Earth
	[4] PSR B1257+12 (Pulsar) - 2313.48 ± 131.685 ly
	[2] Ring Nebula (M57) (Planetary Nebula) - 2569.31 ± 88.744 ly
	[0] Crab Nebula (M1) (Planetary Nebula) - 6383.06 ± 1007.673 ly
	[1] Lagoon Nebula (M8) (Emission Nebula) - 3835.39 ± 434.758 ly
	[3] Alpha Centauri (Star System) - 4.34 ± 0.002 ly
	[nan] Earth


<IPython.core.display.Javascript object>

### Static Route for Further Work

In [11]:
def repr_years(time: u_years) -> str:
    time_days = time * 365
    years = time_days // 365
    time_days %= 365
    months = time_days // 30
    time_days %= 30
    days = time_days

    years = round(years)
    months = round(months)
    days = round(days)
    parts = [(years, "years"), (months, "months"), (days, "days")]
    parts = [(part, unit) for part, unit in parts if part > 0]
    parts = [(f"{part} {unit}" if part != 1 else f"{part} {unit[:-1]}") for part, unit in parts]
    return ", ".join(parts) or "instant"


class CombinedTime:
    def __init__(self, tourist_time: u_years, earth_time: u_years, distance_travelled: u_ly):
        self._tourist_time = tourist_time
        self._earth_time = earth_time
        self._distance_travelled = distance_travelled

    @property
    def tourist_time(self) -> u_years:
        return self._tourist_time

    @property
    def earth_time(self) -> u_years:
        return self._earth_time

    @property
    def distance_travelled(self) -> u_ly:
        return self._distance_travelled

    def __add__(self, other: "CombinedTime") -> "CombinedTime":
        return CombinedTime(self.tourist_time + other.tourist_time, self.earth_time + other.earth_time, self.distance_travelled + other.distance_travelled)

    def __str__(self):
        return f"Tourist Time: {repr_years(self.tourist_time)}, Earth Time: {repr_years(self.earth_time)}, Distance Travelled: {self.distance_travelled:.2f} ly"

    def __repr__(self):
        return f"CombinedTime({self.tourist_time}, {self.earth_time}, {self.distance_travelled})"


def time_between(dest_a: Destination, dest_b: Destination, speed: u_fc) -> CombinedTime:
    dist: u_ly = distance_between(dest_a, dest_b)
    return time_travelled(dist, speed)


def time_travelled(dist: u_ly, speed: u_fc) -> CombinedTime:
    # ly = 2e8 m/s * year
    # ly / (2e8 m/s) = (2e8 m/s * year) / (2e8 m/s) = year

    time_earth_years: u_years = dist / speed
    time_tourist_years: u_years = time_earth_years * np.sqrt(1 - speed**2)
    return CombinedTime(time_tourist_years, time_earth_years, dist)


def low_vel(time: u_years) -> CombinedTime:
    return CombinedTime(time, time, 0)


def weeks_to_years(weeks: float) -> u_years:
    return weeks / 52.1775


static_route = [
    EARTH,
    D_PSR_B1257_12,
    D_RING_NEBULA,
    D_CRAB_NEBULA,
    D_LAGOON_NEBULA,
    D_ALPHA_CENTAURI,
    EARTH
]

assert static_route == best_route_, "Static route does not match best route"

print("Static Route:")
for dest in static_route:
    print(f"\t{dest}")


FC: list[u_fc] = [0.0, 0.9, 0.99, 0.999, 0.9999, 0.9999_9, 0.9999_99, 0.9999_999, 0.9999_9999]
assert FC == sorted(FC), "FC must be sorted"
assert FC[8] == 0.9999_9999

MAX_VEL: u_fc = FC[8]


totals: list[tuple[str, CombinedTime]] = []
def item(name: str, time: CombinedTime):
    global totals
    name += ":"
    print(f"  {name:<40} {time}")
    totals.append((name, time))


times = [time_between(static_route[i], static_route[i + 1], MAX_VEL) for i in range(len(static_route) - 1)]
travel_time = sum(times, CombinedTime(0, 0, 0))

print("\nItinerary:")

item("Main travel", travel_time)

# Other itinerary items
item("Tour through the PSR B1257+12 system", T_PSR_TRANSIT:=low_vel(weeks_to_years(1.5)))
item("Tour across the Ring Nebula", T_RN_TRANSIT:=time_travelled(D_RING_NEBULA.radius, FC[4]))
item("Tour across the Crab Nebula", T_CN_TRANSIT:=time_travelled(D_CRAB_NEBULA.radius, FC[5]))
item("Tour across the Lagoon Nebula", T_LN_TRANSIT:=time_travelled(D_LAGOON_NEBULA.radius, FC[6]))
item("Tour through the Alpha Centauri system", T_AC_TRANSIT:=low_vel(weeks_to_years(1.5)))



print(f"\nTotals: {sum([time for _, time in totals], CombinedTime(0, 0, 0))}")
print(f"Time other than main travel: {sum([time for name, time in totals if 'Main travel' not in name], CombinedTime(0, 0, 0))}")

print("\n\n")

PSR_ARRIVE = time_between(EARTH, D_PSR_B1257_12, MAX_VEL)
PSR_LEAVE = PSR_ARRIVE + T_PSR_TRANSIT
print(f"Arrive at PSR B1257+12: {PSR_ARRIVE}")
print(f"Leave PSR B1257+12: {PSR_LEAVE}")
print()

RN_ARRIVE = PSR_LEAVE + time_between(D_PSR_B1257_12, D_RING_NEBULA, MAX_VEL)
RN_LEAVE = RN_ARRIVE + T_RN_TRANSIT
print(f"Arrive at Ring Nebula: {RN_ARRIVE}")
print(f"Leave Ring Nebula: {RN_LEAVE}")
print()

CN_ARRIVE = RN_LEAVE + time_between(D_RING_NEBULA, D_CRAB_NEBULA, MAX_VEL)
CN_LEAVE = CN_ARRIVE + T_CN_TRANSIT
print(f"Arrive at Crab Nebula: {CN_ARRIVE}")
print(f"Leave Crab Nebula: {CN_LEAVE}")
print()

LN_ARRIVE = CN_LEAVE + time_between(D_CRAB_NEBULA, D_LAGOON_NEBULA, MAX_VEL)
LN_LEAVE = LN_ARRIVE + T_LN_TRANSIT
print(f"Arrive at Lagoon Nebula: {LN_ARRIVE}")
print(f"Leave Lagoon Nebula: {LN_LEAVE}")
print()

AC_ARRIVE = LN_LEAVE + time_between(D_LAGOON_NEBULA, D_ALPHA_CENTAURI, MAX_VEL)
AC_LEAVE = AC_ARRIVE + T_AC_TRANSIT
print(f"Arrive at Alpha Centauri: {AC_ARRIVE}")
print(f"Leave Alpha Centauri: {AC_LEAVE}")
print()

EARTH_ARRIVE = AC_LEAVE + time_between(D_ALPHA_CENTAURI, EARTH, MAX_VEL)
print(f"Arrive back at Earth: {EARTH_ARRIVE}")

Static Route:
	Earth
	PSR B1257+12 (Pulsar) - 2313.48 ± 131.685 ly
	Ring Nebula (M57) (Planetary Nebula) - 2569.31 ± 88.744 ly
	Crab Nebula (M1) (Planetary Nebula) - 6383.06 ± 1007.673 ly
	Lagoon Nebula (M8) (Emission Nebula) - 3835.39 ± 434.758 ly
	Alpha Centauri (Star System) - 4.34 ± 0.002 ly
	Earth

Itinerary:
  Main travel:                             Tourist Time: 2 years, 2 months, 24 days, Earth Time: 15766 years, 10 months, 13 days, Distance Travelled: 15766.86 ly
  Tour through the PSR B1257+12 system:    Tourist Time: 10 days, Earth Time: 10 days, Distance Travelled: 0.00 ly
  Tour across the Ring Nebula:             Tourist Time: 7 days, Earth Time: 1 year, 3 months, 20 days, Distance Travelled: 1.30 ly
  Tour across the Crab Nebula:             Tourist Time: 9 days, Earth Time: 5 years, 6 months, 3 days, Distance Travelled: 5.50 ly
  Tour across the Lagoon Nebula:           Tourist Time: 10 days, Earth Time: 20 years, Distance Travelled: 20.00 ly
  Tour through the Alpha C